In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification

In [3]:

# %timeit sirve para evaluar el tiempo de ejecucion
train_entrenamiento = pd.read_csv('feature_training_paso_uno.csv')
test_entrenamiento = pd.read_csv('feature_test_paso_uno.csv')
test = pd.read_csv('trocafone_kaggle_test.csv')

 

In [4]:
 len(train_entrenamiento)


19414

In [5]:
train_entrenamiento.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,mean_per_event,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,label
0,0008ed71,0.00,0.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,2.00,2.00,checkout,3,generic listing,1,0
1,000c79fe,1.00,0.00,1.00,0.00,1.00,0.00,1.00,9.00,0.00,3.00,1.00,2.43,searched products,9,ad campaign hit,1,0
2,001802e4,5.00,0.00,1.00,0.00,4.00,0.00,0.00,4.00,0.00,4.00,1.00,3.17,ad campaign hit,5,checkout,1,0
3,0019e639,29.00,165.00,15.00,2.00,28.00,0.00,13.00,11.00,0.00,189.00,19.00,52.33,viewed product,189,conversion,2,0
4,001b0bf9,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,2.00,0.00,1.40,checkout,2,brand listing,1,0


In [6]:
test_entrenamiento.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,mean_per_event,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event
0,4886f805,0.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,0.00,4.00,1.00,1.50,viewed product,4,checkout,1
1,0297fc1e,29.00,4.00,7.00,0.00,21.00,1.00,0.00,6.00,0.00,404.00,95.00,70.88,viewed product,404,lead,1
2,2d681dd8,1.00,5.00,1.00,0.00,1.00,0.00,2.00,1.00,0.00,13.00,2.00,3.25,viewed product,13,ad campaign hit,1
3,cccea85e,15.00,7.00,1.00,0.00,20.00,0.00,26.00,1.00,5.00,739.00,22.00,92.89,viewed product,739,checkout,1
4,4c8a8b93,14.00,8.00,2.00,0.00,14.00,0.00,13.00,9.00,0.00,177.00,20.00,32.12,viewed product,177,checkout,2


In [7]:
#feature engieri del train,misma columna train,pero sin label.

In [8]:
string_columns=train_entrenamiento.columns
string_columns

Index(['person', 'ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'mean_per_event',
       'most_frequent_event', 'count_most_frequent_event',
       'less_frequent_event', 'count_less_frequent_event', 'label'],
      dtype='object')

In [9]:
#hash
#Transformo texto a numeros
for f in string_columns:
         train_entrenamiento[f] = train_entrenamiento.apply(lambda row:  hash(row[f]) if not(pd.isnull(row[f]))
         else np.nan, axis = 1)

In [10]:
train_entrenamiento.head(5)

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,mean_per_event,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,label
0,-3203356421948216516,0,0,3,0,1,0,0,0,0,0,2,2,9211473729940896763,3,3622496767228748220,1,0
1,1918349594255226024,1,0,1,0,1,0,1,9,0,3,1,988218432520154114,-5153909100894645198,9,-9116664521409518188,1,0
2,6626264205476434721,5,0,1,0,4,0,0,4,0,4,1,384307168202281987,-9116664521409518188,5,9211473729940896763,1,0
3,4689269188416249543,29,165,15,2,28,0,13,11,0,189,19,768614336404586548,3294504626581633526,189,359843521140084213,2,0
4,-6307625735916064315,0,1,2,1,1,0,0,0,0,2,0,922337203685477377,9211473729940896763,2,1980344244791988776,1,0


In [11]:
train_label = train_entrenamiento['label']
train_label.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [12]:
train = train_entrenamiento.drop('label', 1)
train.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,mean_per_event,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event
0,-3203356421948216516,0,0,3,0,1,0,0,0,0,0,2,2,9211473729940896763,3,3622496767228748220,1
1,1918349594255226024,1,0,1,0,1,0,1,9,0,3,1,988218432520154114,-5153909100894645198,9,-9116664521409518188,1
2,6626264205476434721,5,0,1,0,4,0,0,4,0,4,1,384307168202281987,-9116664521409518188,5,9211473729940896763,1
3,4689269188416249543,29,165,15,2,28,0,13,11,0,189,19,768614336404586548,3294504626581633526,189,359843521140084213,2
4,-6307625735916064315,0,1,2,1,1,0,0,0,0,2,0,922337203685477377,9211473729940896763,2,1980344244791988776,1


In [13]:
forest = RandomForestRegressor(n_estimators = 1000, n_jobs = -1,random_state =3 ,
                                max_features = "auto",max_depth=4000, min_samples_leaf = 6)


In [14]:
forest.fit(train, train_label)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4000,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=6, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=False, random_state=3, verbose=0, warm_start=False)

In [15]:
string_columns=test_entrenamiento.columns
string_columns

Index(['person', 'ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'mean_per_event',
       'most_frequent_event', 'count_most_frequent_event',
       'less_frequent_event', 'count_less_frequent_event'],
      dtype='object')

In [16]:
#hash del test
#hash
#Transformo texto a numeros
for f in string_columns:
         test_entrenamiento[f] = test_entrenamiento.apply(lambda row:  hash(row[f]) if not(pd.isnull(row[f]))
         else np.nan, axis = 1)

In [17]:
test_entrenamiento.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,mean_per_event,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event
0,514612251759078211,0,0,1,0,1,0,1,1,0,4,1,1152921504606846977,3294504626581633526,4,9211473729940896763,1
1,343576741627502853,29,4,7,0,21,1,0,6,0,404,95,2017612633061982278,3294504626581633526,404,368424695172741976,1
2,-3655205306433233467,1,5,1,0,1,0,2,1,0,13,2,576460752303423491,3294504626581633526,13,-9116664521409518188,1
3,-3714628536674831697,15,7,1,0,20,0,26,1,5,739,22,2049638230412132444,3294504626581633526,739,9211473729940896763,1
4,1575848761860348290,14,8,2,0,14,0,13,9,0,177,20,288230376151711776,3294504626581633526,177,9211473729940896763,2


In [18]:
#predecimos
prediction = forest.predict(test_entrenamiento)

In [19]:
ids = test['person'].values
dataKaggle = pd.DataFrame(ids ,columns={'person'})
dataKaggle.head(5)

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [20]:
#Aca agregamos una columna que seria se postulop
dataKaggle['label'] = prediction

In [21]:
 dataKaggle.nlargest(5,'label'  )

,person,label
9206,b2542ed6,0.68
7794,62147633,0.62
262,2b213b2c,0.62
13966,845fba39,0.59
804,15135e58,0.59


In [22]:
dataKaggle.to_csv("predictionRandonForest4.csv", index = False)